<a href="https://colab.research.google.com/github/kanchangiri67/OncoCist/blob/main/ML_Model_Loader_for_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio --quiet

# Loss functions

In [ ]:
import tensorflow as tf

@tf.keras.utils.register_keras_serializable()
def iou_metric(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + 1e-7)

@tf.keras.utils.register_keras_serializable()
def dice_loss(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return 1 - (2 * intersection + 1e-7) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-7)

@tf.keras.utils.register_keras_serializable()
def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

# ImagePreprocessor Class

In [ ]:
class ImagePreprocessor:
    @staticmethod
    def normalize(image):
        import numpy as np
        image = np.float64(image)
        min_val, max_val = np.min(image), np.max(image)
        if max_val == min_val:
            return np.zeros_like(image, dtype=np.uint8)
        return np.uint8(255 * (image - min_val) / (max_val - min_val))

In [ ]:
import gradio as gr
import numpy as np
import cv2
import tensorflow as tf

# Load U-Net Model
unet_model = tf.keras.models.load_model(
    "/content/unet_model.keras",
    custom_objects={
        "iou_metric": iou_metric,
        "dice_loss": dice_loss,
        "combined_loss": combined_loss
    }
)

# Load CNN Classifier
cnn_model = tf.keras.models.load_model("/content/cnn_model.keras")
tumor_map = {0: "Meningioma", 1: "Glioma", 2: "Pituitary Tumor"}

# Image inference function
def analyze_mri(image):
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Resize for both models
    resized = cv2.resize(image, (512, 512))
    normalized = ImagePreprocessor.normalize(resized)

    # Predict segmentation
    seg_input = normalized.reshape(1, 512, 512, 1)
    pred_mask = unet_model.predict(seg_input)[0]
    binary_mask = (pred_mask > 0.5).astype(np.uint8).squeeze()

    # Create overlay
    overlay = np.stack([normalized]*3, axis=-1)
    overlay[binary_mask == 1] = [255, 0, 0]  # Red

    # Predict tumor type
    class_input = normalized.reshape(1, 512, 512, 1)
    pred_class = np.argmax(cnn_model.predict(class_input), axis=1)[0]
    tumor_type = tumor_map[pred_class]

    return overlay, tumor_type

# Gradio interface
gr.Interface(
    fn=analyze_mri,
    inputs=gr.Image(label="Upload Brain MRI", type="numpy"),
    outputs=[
        gr.Image(label="Tumor Mask Overlay", type="numpy"),
        gr.Label(label="Predicted Tumor Type")
    ],
    title="Brain Tumor Analysis",
    description="Upload a brain MRI to see the predicted tumor segmentation (U-Net) and classification (CNN)."
).launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://39fbe44fb559a628b5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://39fbe44fb559a628b5.gradio.live
